<a id="title_ID"></a>
# STScI Notebook Tutorial Template/Style Guide
<p style="color:#808080">*This notebook outlines a general consistent framework for STScI tutorials. This is a suggested style guide created under the assumption that there will be exceptions to many of these standards.*</p>

<p style="color:#808080">*Here is a good place to describe the intention of this notebook and what the user will be producing. It should be short and concise (1-2 sentences) to give an idea of what this notebook is about.*</p>

<p style="color:#808080">*Below there is a general Table of Contents outline that can be changed/added to as needed. All notebooks ideally should at least include the following sections: Introduction, Imports, Data Retrieval, Additional Resources, and About this Notebook. Also, while not necessary, it is helpful to include an image (or multiple) of a plot or diagram created in the tutorial as a preview for the user. The NASA and MAST logos acts as placeholders here for this purpose.*</p>
<br>
<img style="float: right;" src="nasa_logo.png" alt="nasa_logo" width="400px"/> 
<img style="float: right;" src="mast_logo.jpg" alt="mast_logo" width="200px"/> 
### Table of Contents
<div style="text-align: left"> <br> [Introduction](#intro_ID) <br> [Imports](#imports_ID) <br> [Getting the Data](#data_ID) <br> [File Information](#fileinfo_ID) <br> [Plotting/Displaying](#plot_ID) <br> [Additional Resources](#resources_ID) <br> [About this Notebook](#about_ID) </div>

***

<a id="intro_ID"></a>
## Introduction

**Some background:**
<p style="color:#808080">*Here is a place to provide some background information about the file, data, plot, celestial object, etc. that is a feature part of this notebook. Again, try to keep this to a few sentences if possible and use external links below to provide more detailed information if necessary.*</p> 

**Defining some terms:**
<p style="color:#808080">*This section is to explain some jargon that might not already be clear to the intended user of this notebook. Brief definitions are ideal with external links to more information provided at the bottom if necessary.* </p>
 - **An example word:** Insert definition here. 
 - **An example acronym:** Insert definition here. 
 - **Another word:** Insert definition here. 

For more information about the {*subject of this notebook*} or {*any of the above definitions*}, visit [{this source}](insert URL here). Feel free to add more links as well if relevant.

[Top of Page](#title_ID)

***

<a id="imports_ID"></a>
## Imports
<p style="color:#808080">*Here is an imports cell example...please keep in mind that the user of this notebook may not have access to many third-party python packages. Make sure to state what the package does for this notebook*</p>

Let's start by importing some libraries to the environment:
 
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib* for plotting data

In [1]:
from astropy.io import fits
from astropy.table import Table 
import matplotlib.pyplot as plt

[Top of Page](#title_ID)

***

<a id="data_ID"></a>
## Getting the Data
<p style="color:#808080">*If the user needs to download data to run the tutorial properly, we suggest using Astroquery to retrieve files. Here is an example query of a specific Kepler light curve file. Depending on the data for your tutorial, you may need to change the way this is queried (for example, by RA and Dec if the region of the sky is the most important trait of the file):*</p>

Start by importing libraries from Astroquery. For a longer, more detailed description using of Astroquery, please visit this [tutorial](https://github.com/spacetelescope/MAST-API-Notebooks/blob/master/MUG2018_APITutorial_Astroquery.ipynb) or read the Astroquery [documentation](https://astroquery.readthedocs.io/en/latest/#). 

In [3]:
from astroquery.mast import Mast
from astroquery.mast import Observations

<br>Next, we need to find the data file. This is similar to searching for the data using the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) in that we will be using certain keywords to find the file. The target name of the object we are looking for is kplr011446443, collected by the Kepler spacecraft.

In [4]:
keplerObs = Observations.query_criteria(target_name='kplr011446443', obs_collection='Kepler')
keplerProds = Observations.get_product_list(keplerObs[1])
yourProd = Observations.filter_products(keplerProds, extension='kplr011446443-2009131110544_slc.fits', 
                                        mrp_only=False)
yourProd

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str6,str10,str36,str60,str1,str110,str9,str28,str1,str1,str6,str1,str7,str44,int64,str10
9000210988,Kepler,timeseries,kplr011446443_sc_Q113313330333033302,Lightcurve Short Cadence (CSC) - Q0,C,mast:Kepler/url/missions/kepler/lightcurves/0114/011446443/kplr011446443-2009131110544_slc.fits,SCIENCE,--,--,--,Kepler,--,EX_STKS,kplr011446443-2009131110544_slc.fits,1457280,9000210988


<br>Now that we've found the data file, we can download it using the reults shown in the table above:

In [5]:
Observations.download_products(yourProd, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str95,str8,object,object
./mastDownload/Kepler/kplr011446443_sc_Q113313330333033302/kplr011446443-2009131110544_slc.fits,COMPLETE,None,None


<br>Click on the blue URL above to download the file. You are now ready to complete the rest of the notebook.

[Top of Page](#title_ID)

***

<a id="fileinfo_ID"></a>
## File Information
<p style="color:#808080">*This is a good place to talk about the data the user has just downloaded through Astroquery. Here is an example of some information related to the file downloaded in the previous section. Please notice the renaming of the data file in the cell below (including the directory and path, which is important to include if the user has just downloaded data from Astroquery).*</p> 
<br>Now that we have the file, we can start working with the data. We will begin by assigning a shorter name to the file to make it easier to use. Then, using the info function from astropy.io.fits, we can see some information about the FITS Header Data Units:

In [6]:
filename = "./mastDownload/Kepler/kplr011446443_sc_Q113313330333033302/kplr011446443-2009131110544_slc.fits"
fits.info(filename)

Filename: ./mastDownload/Kepler/kplr011446443_sc_Q113313330333033302/kplr011446443-2009131110544_slc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   ()      
  1  LIGHTCURVE    1 BinTableHDU    155   14280R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        48   (8, 9)   int32   


 - **No. 0 (Primary): **
 <br>This HDU contains meta-data related to the entire file.
 - **No. 1 (Light curve): **
 <br>This HDU contains a binary table that holds data like flux measurements and times. We will extract information from here when we define the parameters for the light curve plot.
 - **No. 2 (Aperture): **
 <br>This HDU contains the image extension with data collected from the aperture. We will also use this to display a bitmask plot that visually represents the optimal aperture used to create the SAP_FLUX column in HDU1.

For more detailed information about header extensions, look [here](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf#page=17).

[Top of Page](#title_ID)

***

<a id="plot_ID"></a>
## Plotting/Displaying
<br><font color=gray>*Now that the user has downloaded data, they can plot, display or access whatever information is relevant to the tutorial. Depending on the goal of this notebook, this section might be split into several different sections that walk through different steps.*</font>
<br><font color=gray><br>*Some things to consider when writing this section:*</font>

- <font color=gray>*This is main point of the notebook! All the previous parts build up to this and should be referenced as such.*</font>
- <font color=gray>*Make sure to break up code with text when possible into clear steps, using the markdown as explanations for what happens in the code lines.*</font> 
- <font color=gray>*Be clear about where the data is coming from that is being displayed or manipulated (for example, how did you access certain columns of data to make a plot?)*</font>
- <font color=gray>*Try to follow the [PEP8 style guide](https://www.python.org/dev/peps/pep-0008/) as closely as possible.*</font>

[Top of Page](#title_ID)

***

<a id="resources_ID"></a>
## Additional Resources
<p style="color:#808080">*This is a good place to provide the user with further reading (relevant sites, journals, articles, etc.). Here is an example list related to the Kepler data shown earlier:*</p>

For more information about the MAST archive and details about mission data:
<br>
<br>[MAST API](https://mast.stsci.edu/api/v0/index.html)
<br>[Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
<br>[Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
<br>[Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)



***

<a id="about_ID"></a>
## About this Notebook
**Author:** 
<br>**Updated On:** MM/DD/YYYY

***

[Top of Page](#title_ID)
<img style="float: right;" src="stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 